In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install dask
!pip install modin[ray]  # This will install both Modin and Ray
!pip install pandas


In [3]:
import pandas as pd
import time

file_path = '/content/drive/MyDrive/collab output/Parking_Violations_Issued_-_Fiscal_Year_2017.csv'
chunk_size = 10000  # Adjust based on your system's memory

start_time = time.time()
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    pass  # Replace with your processing code
pandas_duration = time.time() - start_time

print(f"Time taken with pandas: {pandas_duration} seconds")


Time taken with pandas: 85.6590371131897 seconds


In [5]:
import dask.dataframe as dd
import time

file_path = '/content/drive/MyDrive/collab output/Parking_Violations_Issued_-_Fiscal_Year_2017.csv'

# Specify data types for problematic columns
dtypes = {'House Number': 'object', 'Time First Observed': 'object'}

start_time = time.time()
df_dask = dd.read_csv(file_path, dtype=dtypes)
df_dask = df_dask.compute()  # Trigger computation
dask_duration = time.time() - start_time

print(f"Time taken with Dask: {dask_duration} seconds")



/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (18,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


Time taken with Dask: 102.61347246170044 seconds


In [10]:
!pip install pandas --upgrade




In [4]:
import modin.pandas as mpd
import ray
import time

# Shutdown any existing Ray instances
ray.shutdown()

# Initialize Ray
ray.init()

# Define the file path
file_path = '/content/drive/MyDrive/collab output/Parking_Violations_Issued_-_Fiscal_Year_2017.csv'

# Start measuring time
start_time = time.time()

# Read the file with Modin
df_modin = mpd.read_csv(file_path)

# Measure the duration
modin_duration = time.time() - start_time

print(f"Time taken with Modin and Ray: {modin_duration} seconds")




2023-11-11 18:53:45,006	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(raylet) [2023-11-11 18:55:44,910 E 36128 36128] (raylet) node_manager.cc:3035: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 424e998c763ef0ddd67f7d3d129f61213cf8a25d4cf6226cce8af27d, IP: 172.28.0.12) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.28.0.12`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
2023

Time taken with Modin and Ray: 259.76805806159973 seconds


In [9]:
import pandas as pd
import time

file_path = '/content/drive/MyDrive/collab output/Parking_Violations_Issued_-_Fiscal_Year_2017.csv'
chunk_size = 10000  # Adjust based on your system's memory

def clean_column_names(df):
    df.columns = df.columns.str.replace('[^A-Za-z0-9]+', '', regex=True).str.strip()
    return df

start_time = time.time()
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunk = clean_column_names(chunk)
    # Add your processing code here for each chunk
pandas_duration = time.time() - start_time

print(f"Time taken with pandas: {pandas_duration} seconds")


Time taken with pandas: 92.33036088943481 seconds


In [13]:
import yaml
import pandas as pd

# Assuming you have your first chunk of DataFrame named 'first_chunk'
first_chunk = next(pd.read_csv(file_path, chunksize=chunk_size))
first_chunk = clean_column_names(first_chunk)

# Define your schema
schema = {
    'separator': '|',
    'columns': first_chunk.columns.tolist()
}

# Write the schema to a YAML file
with open('schema.yaml', 'w') as file:
    yaml.dump(schema, file)

# Print confirmation
print("YAML file created successfully.")



YAML file created successfully.


In [14]:
# Read the YAML file
with open('schema.yaml', 'r') as file:
    loaded_schema = yaml.safe_load(file)

# Validate the number of columns and column names
assert len(loaded_schema['columns']) == len(first_chunk.columns), "Column count does not match."
assert all(loaded_schema['columns'][i] == first_chunk.columns[i] for i in range(len(first_chunk.columns))), "Column names do not match."

# Print confirmation
print("Validation successful - DataFrame matches YAML schema.")


Validation successful - DataFrame matches YAML schema.


In [16]:
#Write File in Pipe-Separated Text File in GZ Format
import pandas as pd

# Define your file paths
input_file_path = '/content/drive/MyDrive/collab output/Parking_Violations_Issued_-_Fiscal_Year_2017.csv'
output_file_path = '/content/drive/MyDrive/collab output/output_file.gz'

# Function to clean column names
def clean_column_names(df):
    df.columns = df.columns.str.replace('[^A-Za-z0-9]+', '', regex=True).str.strip()
    return df

chunk_size = 10000
first_chunk = True

for chunk in pd.read_csv(input_file_path, chunksize=chunk_size):
    chunk = clean_column_names(chunk)
    chunk.to_csv(output_file_path, sep='|', mode='a', index=False, compression='gzip', header=first_chunk)
    first_chunk = False



In [18]:
#Create a Summary of the File
total_rows = 0
num_columns = 0

for chunk in pd.read_csv(input_file_path, chunksize=chunk_size):
    total_rows += len(chunk)
    num_columns = len(chunk.columns)  # This remains constant, so it's enough to set it once

print(f"Total Number of Rows: {total_rows}")
print(f"Total Number of Columns: {num_columns}")


Total Number of Rows: 10803028
Total Number of Columns: 43
